# Limpieza de Prime

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq

import sys, os
sys.path.append(os.path.abspath(".."))

from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [2]:
dfprime_raw = buscar_peliculas('Prime')

<Response [200]> 30 7408


Fecha de consulta en la API: 18-10-2025 23:06

Verificamos que no haya perdida de datos:

In [3]:
largo_dataframe = len(dfprime_raw[0])
largo_segun_API = dfprime_raw[1]
print([largo_dataframe, largo_segun_API])

[7407, 7408]


Los largos coinciden

In [4]:
dfprime1 = dfprime_raw[0]

In [5]:
dfprime1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,11001921,John Candy: I Like Me,2025,tt26683420,1492608,movie,movie
1,1882904,Maintenance Required,2025,tt33335602,1352755,movie,movie
2,1535445,Gladiator II,2024,tt9218128,558449,movie,movie
3,1532981,The Batman,2022,tt1877830,414906,movie,movie
4,1498670,Nosferatu,2024,tt5040012,426063,movie,movie
...,...,...,...,...,...,...,...
7402,1713098,Rage,2023,tt28492037,1129932,movie,movie
7403,1703537,Wolf Hollow,2023,tt14297962,1082791,movie,movie
7404,1696692,Dinosaur Hotel 3,2024,tt22462952,1042049,movie,movie
7405,1686398,Kid Snow,2024,tt21053652,978582,movie,movie


Guardamos estos datos crudos en un csv

In [6]:
dfprime1.to_csv('../data/raw_api_prime.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `type` y `tmdb_type` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [8]:
dfprime1['type'].unique()

array(['movie'], dtype=object)

In [9]:
dfprime1['tmdb_type'].unique()

array(['movie', 'tv'], dtype=object)

In [10]:
dfprime1[dfprime1['tmdb_type'] == 'tv']

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
1557,1670695,Halo Legends,2010,tt1480660,60879,tv,movie


Como no esta separada por capitulos la consideramos una 'movie'. Al ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [11]:
dfprime2 = dfprime1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores Na

In [12]:
resultados = {}
for i in dfprime2.columns:
    resultados[i] = dfprime2[i].isna().sum()
resultados

{'id': np.int64(0),
 'title': np.int64(0),
 'year': np.int64(0),
 'imdb_id': np.int64(0),
 'tmdb_id': np.int64(0)}

### Duplicados

In [13]:
dfprime2.duplicated().sum()

np.int64(19)

In [14]:
dfprime2[dfprime2.duplicated()]

,id,title,year,imdb_id,tmdb_id
251,1462878,Waterworld,1995,tt0114898,9804
750,147479,Ben-Hur,2016,tt2638144,271969
1500,1131368,FearDotCom,2002,tt0295254,9544
2500,1146512,Get Thrashed,2006,tt0780524,29124
3000,1378098,The Bad Kids,2016,tt5278458,376256
3251,11034204,Cop & a Half,2025,tt32440581,1502910
3500,1287438,Legend of the Millennium Dragon,2011,tt1849027,74842
3750,1307736,Pride and Prejudice,2003,tt0366920,18087
4000,1620215,Me and Me,2020,tt12805676,581529
4001,1616732,Ramanaa,2002,tt0378647,78955


In [16]:
dfprime3 = dfprime2.drop_duplicates(keep='first')

#### imdb_id

In [17]:
dfprime3.duplicated('imdb_id').sum()

np.int64(1)

In [19]:
dfprime3[dfprime3.duplicated('imdb_id')]

,id,title,year,imdb_id,tmdb_id
1736,1641857,#Home,2021,tt10534500,859987


In [20]:
dfprime4 = dfprime3.drop_duplicates('imdb_id', keep='first')

#### id
Deberia ser unico para cada pelicula

In [21]:
dfprime4.duplicated('id').sum()

np.int64(0)

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [22]:
dfprime4.duplicated('title').sum()

np.int64(157)

revisemos porque se repiten los titulos de las peliculas

In [24]:
dfprime4[dfprime4.duplicated('title')].head()

,id,title,year,imdb_id,tmdb_id
386,1412522,The Pink Panther,1963,tt0057413,936
422,1278557,Nosferatu,1922,tt0013442,653
602,1291820,Overboard,1987,tt0093693,10780
756,1637486,Goodnight Mommy,2022,tt9000184,814800
875,1624418,Drishyam 2,2021,tt12361178,706872


In [25]:
dfprime4[dfprime4['title'] == 'The Pink Panther']

,id,title,year,imdb_id,tmdb_id
254,1412520,The Pink Panther,2006,tt0383216,12096
386,1412522,The Pink Panther,1963,tt0057413,936


Son distintas peliculas con mismo nombre

### Tipos de columnas

In [26]:
dfprime4.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

Los tipos de las columnas estan bien

### Analisis de columna `year`

In [27]:
dfprime4['year'].describe()

count    7387.000000
mean     2008.387573
std        29.598435
min         0.000000
25%      2005.000000
50%      2016.000000
75%      2020.000000
max      2025.000000
Name: year, dtype: float64

In [28]:
dfprime4[dfprime4['year'] == 0]

,id,title,year,imdb_id,tmdb_id
4164,1925819,Reality Unleashed,0,tt0426557,466704


In [29]:
dfprime4.loc[4164, 'year'] = 2005

In [30]:
dfprime4['year'].describe()

count    7387.000000
mean     2008.658996
std        18.162548
min      1916.000000
25%      2005.000000
50%      2016.000000
75%      2020.000000
max      2025.000000
Name: year, dtype: float64

todos estan dentro de un rango logico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [33]:
dfprime4.to_csv('../data/api_prime.csv', index=False)